In [72]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [73]:
number = '9'

In [74]:
# Load the MNIST dataset
mnist = fetch_openml("mnist_784", version=1)

In [75]:
x = mnist.data
y = mnist.target

# (o dataset já vem embaralhado)

# The pixel intensities are scaled to lie between 0 and 1 
x = x / 255.0

# change y to be 1 when it's 0, and -1 when it's not 0
y = np.where(y == number, 1, -1)

x_train = x[:60000]
y_train = y[:60000]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = x[60000:]
y_test = y[60000:]

x_test = np.array(x_test)
y_test = np.array(y_test)

# add extra column of 1's to x_train and x_test (at the beginning)
x_train = np.insert(x_train, 0, 1, axis=1)
x_test = np.insert(x_test, 0, 1, axis=1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 785) (60000,) (10000, 785) (10000,)


In [76]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [77]:
def sigmoid(z):
    return np.tanh(z)

# Residual function for LM
def residuals(params, X, y, lam):
    beta = params[:-1]
    v = params[-1]
    preds = sigmoid(X @ beta + v)
    residual = preds - y
    # Add regularization
    regularization = lam * beta
    return np.concatenate([residual, regularization])

In [78]:
# Initial parameters
n_features = X_train.shape[1]
initial_params = np.zeros(n_features + 1)
lam = 100 # Regularization parameter

In [79]:
# Solve using Levenberg-Marquardt
result = least_squares(residuals, initial_params, args=(X_train, y_train, lam), verbose=2, method='lm')

`ftol` termination condition is satisfied.
Function evaluations 10239, initial cost 3.0000e+04, final cost 4.6064e+03, first-order optimality 1.43e-01.


In [80]:
print(result.optimality)
print(result.cost)

0.1432416738989669
4606.444464978365


In [81]:
# Extract optimized parameters
beta_opt = result.x[:-1]
v_opt = result.x[-1]

In [82]:
# Predictions
y_hat = np.sign(sigmoid(X_test @ beta_opt + v_opt))

In [83]:
# compare y_hat with y_test
accuracy = np.mean(y_hat == y_test)

print(f'Precisão de {accuracy*100:.2f}%')

Precisão de 96.44%


In [84]:
# save x to weights/number.npy
np.save(f'weights/{number}.npy', result.x)